In [23]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast


In [24]:
from skforecast.drift_detection import RangeDriftDetector
from pprint import pprint
from copy import deepcopy

In [25]:
y = pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D'), name="series_1")
series_df = pd.DataFrame(
    {
        'series_1': y,
        'series_2': y
    }
)
series_df_long = series_df.stack().to_frame().rename(columns={0: 'y'}).swaplevel(0,1).sort_index()
exog = pd.Series(np.random.rand(5000), index=y.index, name="temp")
exog_df = pd.DataFrame(index=y.index)
for i in range(5):
    exog_df[f"feature_{i}"] = np.random.rand(5000)
for i in range(5):
    exog_df[f"cat_feature_{i}"] = np.random.choice(["A", "B", "C"], 5000)


series_dict = series_df.to_dict(orient='series')
exog_dict = {'series_1': exog_df, 'series_2': exog_df}
exog_df_long = [
    exog_df.copy().assign(series_id='series_1'),
    exog_df.copy().assign(series_id='series_2')
]
exog_df_long = pd.concat(exog_df_long).set_index('series_id', append=True).reorder_levels(['series_id', None]).sort_index()

# FIT

In [26]:
range_detector = RangeDriftDetector()
range_detector.fit(y)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(3.931369140042218e-05), np.float64(0.999585187407915))}
['series_1']


In [27]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(3.931369140042218e-05), np.float64(0.999585187407915)),
 'series_2': (np.float64(3.931369140042218e-05), np.float64(0.999585187407915))}
['series_1', 'series_2']


In [28]:
range_detector = RangeDriftDetector()
range_detector.fit(series_dict)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(3.931369140042218e-05), np.float64(0.999585187407915)),
 'series_2': (np.float64(3.931369140042218e-05), np.float64(0.999585187407915))}
['series_1', 'series_2']


In [29]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df_long)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(3.931369140042218e-05), np.float64(0.999585187407915)),
 'series_2': (np.float64(3.931369140042218e-05), np.float64(0.999585187407915))}
['series_1', 'series_2']


In [30]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'temp': (np.float64(0.0002798528939526035), np.float64(0.9997777166483891))}
['temp']


In [31]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'cat_feature_0': {'C', 'A', 'B'},
 'cat_feature_1': {'B', 'A', 'C'},
 'cat_feature_2': {'C', 'A', 'B'},
 'cat_feature_3': {'C', 'A', 'B'},
 'cat_feature_4': {'B', 'A', 'C'},
 'feature_0': (np.float64(0.00010695646121405833),
               np.float64(0.9999978894026635)),
 'feature_1': (np.float64(0.00012130784688813012),
               np.float64(0.9999676749100571)),
 'feature_2': (np.float64(6.210159385011682e-05),
               np.float64(0.9998524477700328)),
 'feature_3': (np.float64(0.00036149280566777353),
               np.float64(0.999982230237919)),
 'feature_4': (np.float64(0.00011343497981008976),
               np.float64(0.9995976353507426))}
['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'cat_feature_0', 'cat_feature_1', 'cat_feature_2', 'cat_feature_3', 'cat_feature_4']


In [32]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_dict)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'C', 'A', 'B'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'C', 'A', 'B'},
              'cat_feature_3': {'C', 'A', 'B'},
              'cat_feature_4': {'B', 'A', 'C'},
              'feature_0': (np.float64(0.00010695646121405833),
                            np.float64(0.9999978894026635)),
              'feature_1': (np.float64(0.00012130784688813012),
                            np.float64(0.9999676749100571)),
              'feature_2': (np.float64(6.210159385011682e-05),
                            np.float64(0.9998524477700328)),
              'feature_3': (np.float64(0.00036149280566777353),
                            np.float64(0.999982230237919)),
              'feature_4': (np.float64(0.00011343497981008976),
                            np.float64(0.9995976353507426))},
 'series_2': {'cat_feature_0': {'C', 'A', 'B'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'C', 'A'

In [33]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df_long)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'C', 'A', 'B'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'C', 'A', 'B'},
              'cat_feature_3': {'C', 'A', 'B'},
              'cat_feature_4': {'B', 'A', 'C'},
              'feature_0': (np.float64(0.00010695646121405833),
                            np.float64(0.9999978894026635)),
              'feature_1': (np.float64(0.00012130784688813012),
                            np.float64(0.9999676749100571)),
              'feature_2': (np.float64(6.210159385011682e-05),
                            np.float64(0.9998524477700328)),
              'feature_3': (np.float64(0.00036149280566777353),
                            np.float64(0.999982230237919)),
              'feature_4': (np.float64(0.00011343497981008976),
                            np.float64(0.9995976353507426))},
 'series_2': {'cat_feature_0': {'C', 'A', 'B'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'C', 'A'

# Predict

In [34]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.series_values_range_)
pprint(range_detector.exog_values_range_)


{'series_1': (np.float64(3.931369140042218e-05), np.float64(0.999585187407915))}
{'temp': (np.float64(0.0002798528939526035), np.float64(0.9997777166483891))}


In [35]:
last_window_valid = y.tail(100).copy()
last_window_invalid = y.tail(100).copy()
last_window_invalid.iloc[5] = 10_000    
range_detector.predict(last_window=last_window_valid)
range_detector.predict(last_window=last_window_invalid)


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'last_window': 'series_1' has one or more values outside the range seen during       │
│ training [0.00004, 0.99959]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :178                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [36]:
range_detector.fit(series_df)
last_window_valid_df = series_df.tail(100).copy()
last_window_invalid_df = series_df.tail(100).copy()
last_window_invalid_df.iloc[1, [0, 1]] = 10_000
display(last_window_invalid_df)
range_detector.predict(last_window=last_window_valid_df)
range_detector.predict(last_window=last_window_invalid_df)

,series_1,series_2
2013-06-01,0.461175,0.461175
2013-06-02,10000.000000,10000.000000
2013-06-03,0.297609,0.297609
2013-06-04,0.296137,0.296137
2013-06-05,0.780779,0.780779
...,...,...
2013-09-04,0.441101,0.441101
2013-09-05,0.397244,0.397244
2013-09-06,0.167735,0.167735
2013-09-07,0.995574,0.995574


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'last_window': 'series_1' has one or more values outside the range seen during       │
│ training [0.00004, 0.99959]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :178                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'last_window': 'series_2' has one or more values outside the range seen during       │
│ training [0.00004, 0.99959]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :178                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [37]:
range_detector.fit(series_dict)
last_window_valid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
last_window_invalid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
for key in last_window_invalid_dict.keys():
    last_window_invalid_dict[key].iloc[1] = 10_000
range_detector.predict(last_window=last_window_valid_dict)
range_detector.predict(last_window=last_window_invalid_dict)

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': 'series_1' has one or more values outside the range seen during training │
│ [0.00004, 0.99959]. This may affect the accuracy of the predictions.                 │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :178                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'series_2' has one or more values outside the range seen during training │
│ [0.00004, 0.99959]. This may affect the accuracy of the predictions.                 │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :178                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [38]:
range_detector.fit(series_df_long)
last_window_valid_df_long= series_df_long.groupby(level=0).tail(100).copy()
last_window_invalid_df_long = series_df_long.groupby(level=0).tail(100).copy()
for key in last_window_invalid_df_long.index.get_level_values(0).unique():
    last_window_invalid_df_long.loc[key, pd.Timestamp("2013-06-02")] = 10_000
range_detector.predict(last_window=last_window_valid_df_long)
range_detector.predict(last_window=last_window_invalid_df_long)

In [39]:
exog_valid = exog.tail(100).copy()
exog_invalid = exog.tail(100).copy()
exog_invalid.iloc[5] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'exog': 'temp' has one or more values outside the range seen during training         │
│ [0.00028, 0.99978]. This may affect the accuracy of the predictions.                 │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :178                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [40]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'exog': 'feature_0' has one or more values outside the range seen during training    │
│ [0.00011, 1.00000]. This may affect the accuracy of the predictions.                 │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :178                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'exog': 'feature_1' has one or more values outside the range seen during training    │
│ [0.00012, 0.99997]. This may affect the accuracy of the predictions.                 │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :178                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [41]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'exog': 'feature_0' has one or more values outside the range seen during training    │
│ [0.00011, 1.00000]. This may affect the accuracy of the predictions.                 │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :178                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'exog': 'feature_1' has one or more values outside the range seen during training    │
│ [0.00012, 0.99997]. This may affect the accuracy of the predictions.                 │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :178                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [42]:
range_detector = RangeDriftDetector()
range_detector.fit(series=series_dict, exog=exog_dict)
exog_valid = deepcopy(exog_dict)
exog_invalid = deepcopy(exog_dict)
for key in exog_invalid.keys():
    
    exog_invalid[key].iloc[1, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': exog 'feature_0' has one or more values outside the range seen during    │
│ training [0.00011, 1.00000]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :178                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': exog 'feature_1' has one or more values outside the range seen during    │
│ training [0.00012, 0.99997]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :178                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': exog 'feature_0' has one or more values outside the range seen during    │
│ training [0.00011, 1.00000]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :178                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': exog 'feature_1' has one or more values outside the range seen during    │
│ training [0.00012, 0.99997]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :178                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯